In [1]:
import pandas as pd
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
import numpy as np
import re
import gensim
import keras
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('datasets/positive.csv', sep=';', names=n)
data_negative = pd.read_csv('datasets/negative.csv', sep=';', names=n)

In [3]:
data_negative = data_negative[0:25000]
print(data_negative.shape[0])
data_negative.head()

25000


,id,date,name,text,typr,rep,rtw,faw,stcount,foll,frien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [4]:
data_positive = data_positive[0:25000]
print(data_positive.shape[0])
data_positive.head()

25000


,id,date,name,text,typr,rep,rtw,faw,stcount,foll,frien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [5]:
data_total = pd.concat([data_negative,data_positive])
data_total.head()

,id,date,name,text,typr,rep,rtw,faw,stcount,foll,frien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [6]:
# длина датасета
len(data_total)

50000

In [7]:
# есть ли какое-либо другое значение, кроме отрицательного и положительного?
data_total['typr'].unique()

array([-1,  1], dtype=int64)

In [8]:
# распределение набора данных
data_total.groupby('typr').nunique()

,id,date,name,text,rep,rtw,faw,stcount,foll,frien,listcount
typr,,,,,,,,,,,
-1,25000,20472,21369,23698,1,115,2,10313,2308,1795,285
1,25000,10386,21039,24307,1,212,3,10985,2708,2034,280


In [9]:
# оставляем только те столбцы, которые будут использоваться
data_total = data_total[['text','typr']]
data_total.head()

,text,typr
0,на работе был полный пиддес :| и так каждое за...,-1
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1
2,@elina_4post как говорят обещаного три года жд...,-1
3,"Желаю хорошего полёта и удачной посадки,я буду...",-1
4,"Обновил за каким-то лешим surf, теперь не рабо...",-1


In [10]:
# есть ли нулевое значение?
data_total["text"].isnull().sum()

0

In [11]:
# заполняем нулевые значения
data_total["text"].fillna("пусто", inplace = True)

In [12]:
# очистки текста
def depure_data(data):
    # удаляем URL
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)
    # удаляем Email
    data = re.sub('\S*@\S*\s?', '', data)
    # удалем симолы новой строки
    data = re.sub('\s+', ' ', data)
    # удаляем одинарные ковычки
    data = re.sub("\'", "", data)

    # приведение к нижнему регистру + замена «ё» на «е»
    # data = data.lower().replace("ё", "е")

    # замена ссылок на токен «URL»
    # data = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', data)

    # замена упоминания пользователя на токен «USER»
    # data = re.sub('@[^\s]+', 'USER', data)

    # удаление знаков пунктуации
    # data = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', data)
    # data = re.sub(' +', ' ', data)

    return data

In [13]:
temp = []
# разделение pd.Series на список
data_to_list = data_total['text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

['на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D:',
 'Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(',
 'как говорят обещаного три года ждут...((',
 'Желаю хорошего полёта и удачной посадки,я буду очень сильно скучать( ',
 'Обновил за каким-то лешим surf, теперь не работает простоплеер :(']

In [14]:
# удаляем повторяющиеся слова из набора данных, следуя идее, что самые короткие и длинные обычно бесполезны
# преобразование строки в список токенов.
# deacc=True удаляет знаки препинания

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(temp))

print(data_words[:10])

[['на', 'работе', 'был', 'полныи', 'пиддес', 'так', 'каждое', 'закрытие', 'месяца', 'же', 'свихнусь', 'так'], ['коллеги', 'сидят', 'рубятся', 'urban', 'terror', 'из', 'за', 'долбаннои', 'винды', 'не', 'могу'], ['как', 'говорят', 'обещаного', 'три', 'года', 'ждут'], ['желаю', 'хорошего', 'полета', 'удачнои', 'посадки', 'буду', 'очень', 'сильно', 'скучать'], ['обновил', 'за', 'каким', 'то', 'лешим', 'surf', 'теперь', 'не', 'работает', 'простоплеер'], ['котенка', 'вчера', 'носик', 'разбила', 'плакала', 'расстраивалась'], ['зашли', 'то', 'он', 'опять', 'затихарился', 'прямо', 'физически', 'страдаю', 'когда', 'он', 'долго', 'молчит'], ['вообще', 'не', 'болею', 'не', 'выздоравливаю'], ['микрофраза', 'учимся', 'срать', 'кирпичами', 'режиме', 'amp', 'quot', 'нон', 'стоп', 'amp', 'quot'], ['хочу', 'тобои', 'помириться', 'но', 'сука', 'гордая', 'никогда', 'этого', 'не', 'сделаю']]


In [15]:
len(data_words)

50000

In [16]:
# функция для детокенизации всех предложений
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [17]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:5])

['на работе был полныи пиддес так каждое закрытие месяца же свихнусь так', 'коллеги сидят рубятся urban terror из за долбаннои винды не могу', 'как говорят обещаного три года ждут', 'желаю хорошего полета удачнои посадки буду очень сильно скучать', 'обновил за каким то лешим surf теперь не работает простоплеер']


In [18]:
data = np.array(data)

In [19]:
labels = np.array(data_total['typr'])

y = []
for i in range(len(labels)):
    #negative
    if labels[i] == -1:
        y.append(0)
    #positive
    if labels[i] == 1:
        y.append(1)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 2, dtype="float32")


In [20]:
len(labels)

50000

In [21]:
# преобразование текстовых данных в двумерные данные с плавающей запятой

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...  780   20   10]
 [   0    0    0 ...   16    1   59]
 [   0    0    0 ...  306  199 1863]
 ...
 [   0    0    0 ...  662 2303 3543]
 [   0    0    0 ...    4  642   20]
 [   0    0    0 ...   30  204 1148]]


In [22]:
print(labels)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [23]:
# разбиение набора данных на обучающую и тестовую выборку в соотношении 4:1

#X_train, X_test, y_train, y_test = train_test_split(tweets, labels, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2, random_state=1)
print (len(X_train),len(X_test),len(y_train),len(y_test))

40000 10000 40000 10000


## RNN типа: двунаправленная LSTM

RNN обрабатывают последовательности, итерируя вдоль элементов последовательности и сохраняя информацию относительно того, что она обрабатывала до сих пор

In [24]:
# Bidirectional LTSM model

# Embedding - слой встраивания
# слой встраивания — это словарь, который связывает целочисленные индексы с плотными векторами, он возвращает 3D-тензор формы с плавающей запятой
# количество возможных токенов - max_words
# размерность встраиваний- 40

# двунаправленный слой
# максимизирует чувствительность порядка RNN: по сути, он состоит из двух RNN (LSTM или GRU), которые обрабатывают входную последовательность в одном другом направлении, чтобы окончательно объединить представления
# другими словами, один из слоев интерпретирует последовательности в хронологическом порядке, а второй — в антихронологическом порядке

# LSTM
# 20 - количество скрытых единиц внутри слоя = размерность выходного пространства
# Dropout - наиболее эффективный и наиболее часто используемый метод регуляризации для сетей и состоит из случайного отключения скрытых единиц во время обучения

# Dense
# 2 - выхода
# softmax в качестве функции активации в конечном слое

# RMSprop / Adam - оптимизатор = это просто механизм, который постоянно вычисляет градиент потерь и определяет, как двигаться против функции потерь, чтобы найти ее глобальные минимумы и, следовательно, найти лучшие параметры сети (ядро модели и веса ее смещения)
# categorical_crossentropy - функция потерь

modelBR = Sequential()
modelBR.add(layers.Embedding(max_words, 40, input_length=max_len))
modelBR.add(layers.Bidirectional(layers.LSTM(20, dropout=0.6)))
modelBR.add(layers.Dense(2, activation='softmax'))
modelBR.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', 
verbose=1, save_best_only=True, mode='auto', period=1, save_weights_only=False)
history = modelBR.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), callbacks=[checkpoint2])

Epoch 1/15
1250/1250 [==============================] - ETA: 0s - loss: 0.6191 - accuracy: 0.6475
Epoch 1: val_accuracy improved from -inf to 0.68990, saving model to best_model2.hdf5
1250/1250 [==============================] - 181s 140ms/step - loss: 0.6191 - accuracy: 0.6475 - val_loss: 0.5776 - val_accuracy: 0.6899
Epoch 2/15
1250/1250 [==============================] - ETA: 0s - loss: 0.5595 - accuracy: 0.7072
Epoch 2: val_accuracy improved from 0.68990 to 0.69200, saving model to best_model2.hdf5
1250/1250 [==============================] - 140s 112ms/step - loss: 0.5595 - accuracy: 0.7072 - val_loss: 0.5800 - val_accuracy: 0.6920
Epoch 3/15
1250/1250 [==============================] - ETA: 0s - loss: 0.5447 - accuracy: 0.7207
Epoch 3: val_accuracy improved from 0.69200 to 0.70590, saving model to best_model2.hdf5
1250/1250 [==============================] - 217s 174ms/step - loss: 0.5447 - accuracy: 0.7207 - val_loss: 0.5582 - val_accuracy: 0.7059
Epoch 4/15
1250/1250 [=========

KeyboardInterrupt: 

In [ ]:
# Single LSTM layer model
# одна модель LSTM (долговременная кратковременная память)

modelSR = Sequential()
modelSR.add(layers.Embedding(max_words, 20))
modelSR.add(layers.LSTM(15, dropout=0.5))
modelSR.add(layers.Dense(2, activation='softmax'))


modelSR.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

# внедрение контрольных точек модели, чтобы сохранить лучшую метрику и не потерять ее при обучении.
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto', period=1, save_weights_only=False)
history = modelSR.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test),callbacks=[checkpoint1])

In [29]:
# best_model = keras.models.load_model("best_model2.hdf5")
best_model = modelBR

In [30]:
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

313/313 - 7s - loss: 0.5500 - accuracy: 0.7170 - 7s/epoch - 23ms/step
Model accuracy:  0.7170000076293945


In [38]:
def NetWork(text):
    sentiment = ['Негативный','Позитивный']
    sequence = tokenizer.texts_to_sequences([text])
    test = pad_sequences(sequence, maxlen=max_len)
    predict = best_model.predict(test)
    print("Вероятность того, что отзыв позитивный:  {:.2f}%".format(predict[0][1] * 100))
    print("Вероятность того, что отзыв негативный:  {:.2f}%".format(predict[0][0] * 100))
    print(sentiment[np.around(predict, decimals=0).argmax(axis=1)[0]])

In [39]:
NetWork('я так рад за тебя')

1/1 [==============================] - 0s 39ms/step
Вероятность того, что отзыв позитивный:  81.88%
Вероятность того, что отзыв негативный:  18.12%
Позитивный


In [40]:
NetWork('какой ужасный день')

1/1 [==============================] - 0s 34ms/step
Вероятность того, что отзыв позитивный:  43.28%
Вероятность того, что отзыв негативный:  56.72%
Негативный


In [44]:
NetWork('какой прекрасный день')

1/1 [==============================] - 0s 73ms/step
Вероятность того, что отзыв позитивный:  43.28%
Вероятность того, что отзыв негативный:  56.72%
Негативный


In [61]:
NetWork('ты сегодня очень плохо выглядешь')

1/1 [==============================] - 0s 59ms/step
Вероятность того, что отзыв позитивный:  11.86%
Вероятность того, что отзыв негативный:  88.13%
Негативный


In [63]:
NetWork('платье, которое ты надела, выглядет плохо')

1/1 [==============================] - 0s 51ms/step
Вероятность того, что отзыв позитивный:  38.44%
Вероятность того, что отзыв негативный:  61.56%
Негативный


In [47]:
NetWork('от тебя очень плохо пахнет')

1/1 [==============================] - 0s 44ms/step
Вероятность того, что отзыв позитивный:  15.66%
Вероятность того, что отзыв негативный:  84.34%
Негативный


In [48]:
NetWork('я думаю, нам пора расстаться')

1/1 [==============================] - 0s 48ms/step
Вероятность того, что отзыв позитивный:  87.16%
Вероятность того, что отзыв негативный:  12.84%
Позитивный


In [49]:
NetWork('сегодня какой-то нехороший день у меня')

1/1 [==============================] - 0s 41ms/step
Вероятность того, что отзыв позитивный:  18.54%
Вероятность того, что отзыв негативный:  81.46%
Негативный


In [50]:
NetWork('Как понизить температуру ноута?')

1/1 [==============================] - 0s 44ms/step
Вероятность того, что отзыв позитивный:  36.92%
Вероятность того, что отзыв негативный:  63.08%
Негативный


In [52]:
NetWork('сегодня будет зачет по нейронным сетям')

1/1 [==============================] - 0s 52ms/step
Вероятность того, что отзыв позитивный:  20.66%
Вероятность того, что отзыв негативный:  79.34%
Негативный


In [53]:
NetWork('моя нейронная сеть не обучается')

1/1 [==============================] - 0s 44ms/step
Вероятность того, что отзыв позитивный:  32.49%
Вероятность того, что отзыв негативный:  67.51%
Негативный


In [57]:
NetWork('моя нейронная сеть успешно обучится')

1/1 [==============================] - 0s 38ms/step
Вероятность того, что отзыв позитивный:  62.16%
Вероятность того, что отзыв негативный:  37.84%
Позитивный


In [60]:
NetWork('сегодня ясный день')

1/1 [==============================] - 0s 42ms/step
Вероятность того, что отзыв позитивный:  39.91%
Вероятность того, что отзыв негативный:  60.09%
Негативный
